In [13]:
import pandas as pd 
import numpy as np

In [14]:
df = pd.read_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/HEFTcom24/data/solar1.csv")

In [15]:
df = df.drop(df.columns[0], axis= 1)
df.sort_values(by= "valid_time")
df = df.groupby("valid_time").tail(1).reset_index(drop= True)
df["valid_time"] = pd.to_datetime(df["valid_time"])

In [16]:
mask = (df["reference_time"] >= "2022-11-24") & (df["reference_time"] <= "2022-12-07")

dwd_cols = [
    'CloudCover_Point0_dwd', 'CloudCover_Point1_dwd', 'CloudCover_Point2_dwd',
    'CloudCover_Point3_dwd', 'CloudCover_Point4_dwd', 'CloudCover_Point5_dwd',
    'CloudCover_Point6_dwd', 'CloudCover_Point7_dwd', 'CloudCover_Point8_dwd',
    'CloudCover_Point9_dwd', 'CloudCover_Point10_dwd', 'CloudCover_Point11_dwd',
    'CloudCover_Point12_dwd', 'CloudCover_Point13_dwd', 'CloudCover_Point14_dwd',
    'CloudCover_Point15_dwd', 'CloudCover_Point16_dwd', 'CloudCover_Point17_dwd',
    'CloudCover_Point18_dwd', 'CloudCover_Point19_dwd', 'SolarDownwardRadiation_Point0_dwd',
    'SolarDownwardRadiation_Point1_dwd', 'SolarDownwardRadiation_Point2_dwd',
    'SolarDownwardRadiation_Point3_dwd', 'SolarDownwardRadiation_Point4_dwd',
    'SolarDownwardRadiation_Point5_dwd', 'SolarDownwardRadiation_Point6_dwd',
    'SolarDownwardRadiation_Point7_dwd', 'SolarDownwardRadiation_Point8_dwd',
    'SolarDownwardRadiation_Point9_dwd', 'SolarDownwardRadiation_Point10_dwd',
    'SolarDownwardRadiation_Point11_dwd', 'SolarDownwardRadiation_Point12_dwd',
    'SolarDownwardRadiation_Point13_dwd', 'SolarDownwardRadiation_Point14_dwd',
    'SolarDownwardRadiation_Point15_dwd', 'SolarDownwardRadiation_Point16_dwd',
    'SolarDownwardRadiation_Point17_dwd', 'SolarDownwardRadiation_Point18_dwd',
    'SolarDownwardRadiation_Point19_dwd', 'Temperature_Point0_dwd',
       'Temperature_Point1_dwd', 'Temperature_Point2_dwd',
       'Temperature_Point3_dwd', 'Temperature_Point4_dwd',
       'Temperature_Point5_dwd', 'Temperature_Point6_dwd',
       'Temperature_Point7_dwd', 'Temperature_Point8_dwd',
       'Temperature_Point9_dwd', 'Temperature_Point10_dwd',
       'Temperature_Point11_dwd', 'Temperature_Point12_dwd',
       'Temperature_Point13_dwd', 'Temperature_Point14_dwd',
       'Temperature_Point15_dwd', 'Temperature_Point16_dwd',
       'Temperature_Point17_dwd', 'Temperature_Point18_dwd',
       'Temperature_Point19_dwd', 'Temperature_Point0_ncep',
       'Temperature_Point1_ncep', 'Temperature_Point2_ncep',
       'Temperature_Point3_ncep', 'Temperature_Point4_ncep',
       'Temperature_Point5_ncep', 'Temperature_Point6_ncep',
       'Temperature_Point7_ncep', 'Temperature_Point8_ncep',
       'Temperature_Point9_ncep', 'Temperature_Point10_ncep',
       'Temperature_Point11_ncep', 'Temperature_Point12_ncep',
       'Temperature_Point13_ncep', 'Temperature_Point14_ncep',
       'Temperature_Point15_ncep', 'Temperature_Point16_ncep',
       'Temperature_Point17_ncep', 'Temperature_Point18_ncep',
       'Temperature_Point19_ncep'
]

for dwd_col in dwd_cols:
    ncep_col = dwd_col.replace("dwd", "ncep")
    if ncep_col in df.columns:
        df.loc[mask, dwd_col] = df.loc[mask, ncep_col]

## Features

In [17]:
def pv_temperature_efficiency(irradiance, ambient_temp, NOCT=45, wind_speed=1, eta_0=0.18, beta=0.004):
    """
    Calculate the PV cell temperature and efficiency based on solar irradiance and ambient temperature.
    """
    
    Tc = ambient_temp + (NOCT - 20) * (irradiance / 800)
    
    efficiency = eta_0 * (1 - beta * (Tc - 25))
    
    return Tc, efficiency

In [18]:
temperature_columns = [
    'Temperature_Point0_dwd',
       'Temperature_Point1_dwd', 'Temperature_Point2_dwd',
       'Temperature_Point3_dwd', 'Temperature_Point4_dwd',
       'Temperature_Point5_dwd', 'Temperature_Point6_dwd',
       'Temperature_Point7_dwd', 'Temperature_Point8_dwd',
       'Temperature_Point9_dwd', 'Temperature_Point10_dwd',
       'Temperature_Point11_dwd', 'Temperature_Point12_dwd',
       'Temperature_Point13_dwd', 'Temperature_Point14_dwd',
       'Temperature_Point15_dwd', 'Temperature_Point16_dwd',
       'Temperature_Point17_dwd', 'Temperature_Point18_dwd',
       'Temperature_Point19_dwd', 'Temperature_Point0_ncep',
       'Temperature_Point1_ncep', 'Temperature_Point2_ncep',
       'Temperature_Point3_ncep', 'Temperature_Point4_ncep',
       'Temperature_Point5_ncep', 'Temperature_Point6_ncep',
       'Temperature_Point7_ncep', 'Temperature_Point8_ncep',
       'Temperature_Point9_ncep', 'Temperature_Point10_ncep',
       'Temperature_Point11_ncep', 'Temperature_Point12_ncep',
       'Temperature_Point13_ncep', 'Temperature_Point14_ncep',
       'Temperature_Point15_ncep', 'Temperature_Point16_ncep',
       'Temperature_Point17_ncep', 'Temperature_Point18_ncep',
       'Temperature_Point19_ncep',
]

irradiance_columns = [
    'SolarDownwardRadiation_Point0_dwd', 'SolarDownwardRadiation_Point1_dwd',
    'SolarDownwardRadiation_Point2_dwd', 'SolarDownwardRadiation_Point3_dwd',
    'SolarDownwardRadiation_Point4_dwd', 'SolarDownwardRadiation_Point5_dwd',
    'SolarDownwardRadiation_Point6_dwd', 'SolarDownwardRadiation_Point7_dwd',
    'SolarDownwardRadiation_Point8_dwd', 'SolarDownwardRadiation_Point9_dwd',
    'SolarDownwardRadiation_Point10_dwd', 'SolarDownwardRadiation_Point11_dwd',
    'SolarDownwardRadiation_Point12_dwd', 'SolarDownwardRadiation_Point13_dwd',
    'SolarDownwardRadiation_Point14_dwd', 'SolarDownwardRadiation_Point15_dwd',
    'SolarDownwardRadiation_Point16_dwd', 'SolarDownwardRadiation_Point17_dwd',
    'SolarDownwardRadiation_Point18_dwd', 'SolarDownwardRadiation_Point19_dwd',
    'SolarDownwardRadiation_Point0_ncep', 'SolarDownwardRadiation_Point1_ncep',
    'SolarDownwardRadiation_Point2_ncep', 'SolarDownwardRadiation_Point3_ncep',
    'SolarDownwardRadiation_Point4_ncep', 'SolarDownwardRadiation_Point5_ncep',
    'SolarDownwardRadiation_Point6_ncep', 'SolarDownwardRadiation_Point7_ncep',
    'SolarDownwardRadiation_Point8_ncep', 'SolarDownwardRadiation_Point9_ncep',
    'SolarDownwardRadiation_Point10_ncep', 'SolarDownwardRadiation_Point11_ncep',
    'SolarDownwardRadiation_Point12_ncep', 'SolarDownwardRadiation_Point13_ncep',
    'SolarDownwardRadiation_Point14_ncep', 'SolarDownwardRadiation_Point15_ncep',
    'SolarDownwardRadiation_Point16_ncep', 'SolarDownwardRadiation_Point17_ncep',
    'SolarDownwardRadiation_Point18_ncep', 'SolarDownwardRadiation_Point19_ncep'
]

for i in range(20):
    for source in ['dwd', 'ncep']:
        temp_col = f'Temperature_Point{i}_{source}'
        irradiance_col = f'SolarDownwardRadiation_Point{i}_{source}'
        panel_temp_col = f'Panel_Temperature_Point{i}_{source}'
        panel_eff_col = f'Panel_Efficiency_Point{i}_{source}'
        
        df[panel_temp_col], df[panel_eff_col] = zip(*df.apply(
            lambda row: pv_temperature_efficiency(row[irradiance_col], row[temp_col]), axis=1))

In [19]:
df["SolarDownwardRadiation_Mean"] = df.filter(like= "SolarDownwardRadiation").mean(axis= 1)
df["SolarDownwardRadiation_ncep_Mean"] = df.filter(regex= r"SolarDownwardRadiation.*_ncep").mean(axis= 1)
df["SolarDownwardRadiation_dwd_Mean"] = df.filter(regex= r"SolarDownwardRadiation.*_dwd").mean(axis= 1)
df["SolarDownwardRadiation_RW_Mean_1h"] = df["SolarDownwardRadiation_Mean"].rolling(window= 2).mean()
df["SolarDownwardRadiation_RW_dwd_Mean_30min"] = df["SolarDownwardRadiation_dwd_Mean"].rolling(window= 1).mean()
df["SolarDownwardRadiation_RW_dwd_Mean_1h"] = df["SolarDownwardRadiation_dwd_Mean"].rolling(window= 2).mean()
df["SolarDownwardRadiation_dwd_Mean_Lag_30min"] = df["SolarDownwardRadiation_dwd_Mean"].shift(periods= 1)
df["SolarDownwardRadiation_Mean_Lag_1h"] = df["SolarDownwardRadiation_Mean"].shift(periods= 2)
df["SolarDownwardRadiation_Mean_Lag_24h"] = df["SolarDownwardRadiation_Mean"].shift(periods= 48)
df["Panel_Efficiency_Mean"] = df.filter(regex= r"Panel_Efficiency.*").mean(axis= 1)
df["Panel_Efficiency_Std"] = df.filter(regex= r"Panel_Efficiency.*").std(axis= 1)
df["Panel_Temperature_Mean"] = df.filter(regex= r"Panel_Temperature.*").mean(axis= 1)
df["Panel_Temperature_Std"] = df.filter(regex= r"Panel_Temperature.*").std(axis= 1)
df["Temperature_dwd_Std"] = df.filter(regex= r"Temperature.*_dwd").std(axis= 1)
df["Temperature_dwd_Mean"] = df.filter(regex= r"Temperature.*_dwd").mean(axis= 1)
df["sin_hour"] = np.sin(2 * np.pi * df["valid_time"].dt.hour / 24)
df["cos_hour"] = np.cos(2 * np.pi * df["valid_time"].dt.hour / 24)
df["sin_day"] = np.sin(2 * np.pi * df["valid_time"].dt.dayofyear / 365)
df["cos_day"] = np.cos(2 * np.pi * df["valid_time"].dt.dayofyear / 365)
df["Solar_MWh_Lag_24h"] = df["Solar_MWh_credit"].shift(periods= 48)

/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_8450/212981314.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["cos_day"] = np.cos(2 * np.pi * df["valid_time"].dt.dayofyear / 365)
/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_8450/212981314.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["Solar_MWh_Lag_24h"] = df["Solar_MWh_credit"].shift(periods= 48)


In [20]:
df = df.sort_values(by= "valid_time")

In [21]:
features = [
    "SolarDownwardRadiation_Mean",
    #"SolarDownwardRadiation_ncep_Mean",
    #"SolarDownwardRadiation_dwd_Mean",
    "SolarDownwardRadiation_RW_Mean_1h",
    "SolarDownwardRadiation_RW_dwd_Mean_30min",
    #"SolarDownwardRadiation_RW_dwd_Mean_1h",
    "SolarDownwardRadiation_dwd_Mean_Lag_30min",
    "SolarDownwardRadiation_Mean_Lag_1h",
    "SolarDownwardRadiation_Mean_Lag_24h",
    "Panel_Efficiency_Mean",
    "Panel_Efficiency_Std",
    "Panel_Temperature_Mean",
    "Panel_Temperature_Std",
    "Temperature_dwd_Std",
    "Temperature_dwd_Mean",
    "cos_hour",
    "cos_day",
    "Solar_MWh_Lag_24h"
    ]

df = df[features + ["Solar_MWh_credit"]]

In [22]:
df.columns

Index(['SolarDownwardRadiation_Mean', 'SolarDownwardRadiation_RW_Mean_1h',
       'SolarDownwardRadiation_RW_dwd_Mean_30min',
       'SolarDownwardRadiation_dwd_Mean_Lag_30min',
       'SolarDownwardRadiation_Mean_Lag_1h',
       'SolarDownwardRadiation_Mean_Lag_24h', 'Panel_Efficiency_Mean',
       'Panel_Efficiency_Std', 'Panel_Temperature_Mean',
       'Panel_Temperature_Std', 'Temperature_dwd_Std', 'Temperature_dwd_Mean',
       'cos_hour', 'cos_day', 'Solar_MWh_Lag_24h', 'Solar_MWh_credit'],
      dtype='object')

In [23]:
df.to_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/Generation_forecast/Solar_forecast/data/train.csv", index= False)